In [2]:
import numpy as np

In [3]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [4]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [5]:
CONSTRAINTS

array([[314, 168],
       [212, 157],
       [112,  69]])

In [6]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

In [7]:
solution

array([[False, False, False, False, False, False, False,  True,  True,
        False],
       [False,  True, False, False, False,  True,  True,  True, False,
        False],
       [ True,  True,  True,  True, False,  True, False,  True,  True,
        False]])

In [8]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.False_

In [9]:
# Check if the solution is valid
all_knapsacks = np.any(solution, axis=0)
np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

np.False_

## TEST PROBLEMS

In [36]:
#problem 1
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100*NUM_ITEMS//NUM_KNAPSACKS, size=NUM_DIMENSIONS)

# Max possible value is 20 items * 100 value per item = 2000.
# We set a large PENALTY to ensure infeasible solutions score poorly.
PENALTY = 20000
MAX_ITERATIONS = 500 # A safety limit for the search loop

print(f"Knapsack Constraints: {CONSTRAINTS}")
print("-" * 50)

# ------------------------------------------------------------------------------------------------------
# FITNESS FUNCTION

def calculate_fitness(selection_list, values, weights, constraints, penalty):
    """
    Calculates the fitness score: Total Value - (Penalty * Total Violation Amount).
    """
    # 1. Calculate the value
    total_value = np.sum(values[selection_list == 1])

    # 2. Calculate current weights and violation
    current_weights = np.sum(weights[selection_list == 1], axis=0)

    # Violation is the amount by which weight exceeds capacity (0 if no violation)
    violation = np.maximum(0, current_weights - constraints)
    total_violation_amount = np.sum(violation)

    # 3. Calculate the penalized fitness score
    fitness_score = total_value - (penalty * total_violation_amount)

    return fitness_score, total_value, current_weights

# -----------------------------------------------------------------------------------------------------------
# 1. INITIAL SOLUTION (Start with an empty knapsack)

# our current_selection is a binary vector (0=not selected, 1=selected)
current_selection = np.zeros(NUM_ITEMS, dtype=int)

# Calculate initial fitness
current_fitness, current_value, current_weights = calculate_fitness(
    current_selection, VALUES, WEIGHTS, CONSTRAINTS, PENALTY
)

print(f"Initial Fitness (Empty Knapsack): {current_fitness}")
print("-" * 50)

# ------------------------------------------------------------------------------------
#  HILL CLIMBING (Local Search)

has_improved = True
iteration = 0

# Loop until no move can be found that improves the fitness score
while has_improved and iteration < MAX_ITERATIONS:
    has_improved = False

    # Keep track of the best move found in this iteration
    best_neighbor_selection = current_selection.copy()
    best_neighbor_fitness = current_fitness

    # Pre-calculate indices for efficiency
    in_knapsack = np.where(current_selection == 1)[0]
    out_knapsack = np.where(current_selection == 0)[0]

    # --- Explore Neighborhood (ADD, DROP, SWAP) ---

    # A. Try all possible 'ADD' moves
    for i in out_knapsack:
        test_selection = current_selection.copy()
        test_selection[i] = 1 # Try to ADD item i

        test_fitness, _, _ = calculate_fitness(test_selection, VALUES, WEIGHTS, CONSTRAINTS, PENALTY)

        if test_fitness > best_neighbor_fitness:
            best_neighbor_fitness = test_fitness
            best_neighbor_selection = test_selection.copy()
            has_improved = True

    # B. Try all possible 'DROP' moves
    for i in in_knapsack:
        test_selection = current_selection.copy()
        test_selection[i] = 0 # Try to DROP item i

        test_fitness, _, _ = calculate_fitness(test_selection, VALUES, WEIGHTS, CONSTRAINTS, PENALTY)

        if test_fitness > best_neighbor_fitness:
            best_neighbor_fitness = test_fitness
            best_neighbor_selection = test_selection.copy()
            has_improved = True

    # C. Try all possible 'SWAP' moves
    for i in in_knapsack:
        for k in out_knapsack:
            test_selection = current_selection.copy()
            test_selection[i] = 0 # Drop item i
            test_selection[k] = 1 # Add item k

            test_fitness, _, _ = calculate_fitness(test_selection, VALUES, WEIGHTS, CONSTRAINTS, PENALTY)

            if test_fitness > best_neighbor_fitness:
                best_neighbor_fitness = test_fitness
                best_neighbor_selection = test_selection.copy()
                has_improved = True

    # --- Move to the best neighbor (The "Climb") ---
    if has_improved:
        current_selection = best_neighbor_selection
        current_fitness = best_neighbor_fitness

    iteration += 1

    #---------------------------------------------------------------------------------------------------------------------

# FINAL RESULTS

final_fitness, final_value, final_weights = calculate_fitness(
    current_selection, VALUES, WEIGHTS, CONSTRAINTS, PENALTY
)
selected_item_indices = np.where(current_selection == 1)[0]

if final_fitness < final_value:
    print(f" **INFEASIBLE SOLUTION**: Constraints were violated (Fitness < Value).")
    print(f"   Violation is penalized, true value is {final_value}.")
else:
    print(" **FEASIBLE SOLUTION**.")

print(f"Iterations run: {iteration}")
print("--- Final Results ---")
print(f"Final Fitness: {final_fitness}")


print(f"Final Total Value: {final_value} ")
print(f"Final Total Weights: {final_weights}")
print(f"Knapsack Constraints: {CONSTRAINTS}")
print(f"Selected Item Indices: {selected_item_indices}")

Knapsack Constraints: [614 496]
--------------------------------------------------
Initial Fitness (Empty Knapsack): 0
--------------------------------------------------
 **FEASIBLE SOLUTION**.
Iterations run: 11
--- Final Results ---
Final Fitness: 752
Final Total Value: 752 
Final Total Weights: [521 496]
Knapsack Constraints: [614 496]
Selected Item Indices: [ 1  4  5  7 11 12 13 14 15 18]


In [35]:
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000*2, 1000*NUM_ITEMS//NUM_KNAPSACKS, size=NUM_DIMENSIONS)

PENALTY = 1000 * NUM_ITEMS * 10 # Safe penalty: Max Value * 10
MAX_LOCAL_SEARCH_ITERATIONS = 200 # Max steps for one Hill Climb run
NUM_STARTS = 5  # Run the local search 5 times from different starting points

print(f"Items: {NUM_ITEMS}, Dimensions: {NUM_DIMENSIONS}")
print(f"Knapsack Constraints: {CONSTRAINTS}")
print("-" * 50)

# -------------------------------------------------------------------------------------------

def calculate_fitness(selection, values, weights, constraints, penalty):
    """Calculates fitness: Total Value - Penalty * Total Violation."""
    total_value = np.sum(values[selection == 1])
    current_weights = np.sum(weights[selection == 1], axis=0)
    violation = np.maximum(0, current_weights - constraints)
    total_violation_amount = np.sum(violation)
    fitness_score = total_value - (penalty * total_violation_amount)
    return fitness_score, total_value, current_weights
 # --------------------------------------------------------------------------------------------

def hill_climbing_search(initial_selection, values, weights, constraints, penalty, max_iters):
    """Performs the local search (Hill Climbing) from an initial solution."""
    current_selection = initial_selection.copy()
    current_fitness, _, _ = calculate_fitness(current_selection, values, weights, constraints, penalty)

    for _ in range(max_iters):
        best_neighbor_selection = current_selection.copy()
        best_neighbor_fitness = current_fitness
        improvement_found = False

        in_knapsack = np.where(current_selection == 1)[0]
        out_knapsack = np.where(current_selection == 0)[0]

        # --- Local Search Moves (Focus on ADD and SWAP for faster convergence) ---

        # A. Try all possible 'ADD' moves
        for i in out_knapsack:
            test_selection = current_selection.copy()
            test_selection[i] = 1
            test_fitness, _, _ = calculate_fitness(test_selection, values, weights, constraints, penalty)
            if test_fitness > best_neighbor_fitness:
                best_neighbor_fitness = test_fitness
                best_neighbor_selection = test_selection.copy()
                improvement_found = True

        # B. Try all possible 'SWAP' moves
        # This is the most computationally expensive part, but crucial for MKP
        for i in in_knapsack:
            for k in out_knapsack:
                test_selection = current_selection.copy()
                test_selection[i] = 0 # Drop
                test_selection[k] = 1 # Add
                test_fitness, _, _ = calculate_fitness(test_selection, values, weights, constraints, penalty)
                if test_fitness > best_neighbor_fitness:
                    best_neighbor_fitness = test_fitness
                    best_neighbor_selection = test_selection.copy()
                    improvement_found = True

        # Move to the best neighbor
        if improvement_found and best_neighbor_fitness > current_fitness:
            current_selection = best_neighbor_selection
            current_fitness = best_neighbor_fitness
        else:
            break # No improvement found, reached local optimum

    return current_selection, current_fitness

#-----------------------------------------------------------------------------------------------------------------

# 3. MULTI-START EXECUTION

# This is what we store once the best fitness is found.
best_global_selection = np.zeros(NUM_ITEMS, dtype=int)
best_global_fitness = -np.inf
# By tracking the highest fitness, we ensure we always prioritize solutions that are either feasible and high-value, or less infeasible than others.

print(f"Starting Multi-Start Local Search with {NUM_STARTS} runs from an empty knapsack...")

# Define the single, simple initial solution: the empty knapsack
empty_knapsack = np.zeros(NUM_ITEMS, dtype=int)

for run in range(NUM_STARTS):
    print(f"Starting run {run+1}...")

    # Start the Hill Climbing search from the empty knapsack
    final_solution, final_fitness = hill_climbing_search(
        empty_knapsack, VALUES, WEIGHTS, CONSTRAINTS, PENALTY, MAX_LOCAL_SEARCH_ITERATIONS
    )

    # Track the best result
    if final_fitness > best_global_fitness:
        best_global_fitness = final_fitness
        best_global_selection = final_solution.copy()

    final_value, _, _ = calculate_fitness(final_solution, VALUES, WEIGHTS, CONSTRAINTS, PENALTY)
    print(f"Run {run+1} completed. Final Value: {final_value}")
# -------------------------------------------------------------------------------------------------------------------

# 4. FINAL RESULTS

final_fitness, final_value, final_weights = calculate_fitness(
    best_global_selection, VALUES, WEIGHTS, CONSTRAINTS, PENALTY
)
selected_item_indices = np.where(best_global_selection == 1)[0]

if final_fitness < final_value:
    print(f" **INFEASIBLE SOLUTION**: Constraints were violated (Fitness < Value).")
    print(f"   Violation is penalized, true value is {final_value}.")
else:
    print(" **FEASIBLE SOLUTION**.")

print("--- Overall Best Solution Found (Local Optimum) ---")
print(f"Final Total Value: {final_value}")
print(f"Final Total Weights: {final_weights}")
print(f"Knapsack Constraints: {CONSTRAINTS}")
print(f"Selected Item Count: {len(selected_item_indices)}")


Items: 100, Dimensions: 10
Knapsack Constraints: [7642 9489 9148 2255 9736 6477 6811 3893 2859 2271]
--------------------------------------------------
Starting Multi-Start Local Search with 5 runs from an empty knapsack...
Starting run 1...
Run 1 completed. Final Value: 3909
Starting run 2...
Run 2 completed. Final Value: 3909
Starting run 3...
Run 3 completed. Final Value: 3909
Starting run 4...
Run 4 completed. Final Value: 3909
Starting run 5...
Run 5 completed. Final Value: 3909
 **FEASIBLE SOLUTION**.
--- Overall Best Solution Found (Local Optimum) ---
Final Total Value: 3909
Final Total Weights: [1483 1738 2537 2020 2521 2224 3233 2663 1844 2265]
Knapsack Constraints: [7642 9489 9148 2255 9736 6477 6811 3893 2859 2271]
Selected Item Count: 5


In [34]:
# Problem 3
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000*10, 1000*2*NUM_ITEMS//NUM_KNAPSACKS, size=NUM_DIMENSIONS)

# Safe penalty: Max Value * 10 (1000 * 5000 * 10 = 50,000,000)
PENALTY = 1000 * NUM_ITEMS * 10
MAX_LOCAL_SEARCH_ITERATIONS = 400 # Max steps for the search run
# The size of the neighborhood to check per iteration. Crucial for speed.
SAMPLE_SIZE = 5000

print(f"Items: {NUM_ITEMS}, Dimensions: {NUM_DIMENSIONS}")
print(f"Knapsack Constraints (First 5): {CONSTRAINTS[:5]}...")
print(f"Neighborhood Sample Size: {SAMPLE_SIZE:,}")
print("-" * 70)

# --------------------------------------------------------------------------------------------------------

def calculate_fitness(selection, values, weights, constraints, penalty):
    """Calculates fitness: Total Value - Penalty * Total Violation."""
    total_value = np.sum(values[selection == 1])
    current_weights = np.sum(weights[selection == 1], axis=0)
    violation = np.maximum(0, current_weights - constraints)
    total_violation_amount = np.sum(violation)
    fitness_score = total_value - (penalty * total_violation_amount)
    return fitness_score, total_value, current_weights

def stochastic_hill_climbing(initial_selection, values, weights, constraints, penalty, max_iters, sample_size):
    """Performs Hill Climbing by checking only a random sample of neighbors."""
    current_selection = initial_selection.copy()
    current_fitness, _, _ = calculate_fitness(current_selection, values, weights, constraints, penalty)

    for _ in range(max_iters):
        best_neighbor_selection = current_selection.copy()
        best_neighbor_fitness = current_fitness
        improvement_found = False

        # --- Stochastic Neighborhood Sampling ---
        for _ in range(sample_size):
            in_knapsack = np.where(current_selection == 1)[0]
            out_knapsack = np.where(current_selection == 0)[0]

            # Select move type (ADD, DROP, SWAP) ensuring valid index choices
            if len(in_knapsack) == 0:
                move_type = 0 # Must ADD
            elif len(out_knapsack) == 0:
                move_type = 1 # Must DROP
            else:
                move_type = rng.integers(0, 3)

            test_selection = current_selection.copy()

            # Execute the random move
            if move_type == 0: # ADD move
                i = rng.choice(out_knapsack)
                test_selection[i] = 1
            elif move_type == 1: # DROP move
                i = rng.choice(in_knapsack)
                test_selection[i] = 0
            else: # SWAP move
                i = rng.choice(in_knapsack)
                k = rng.choice(out_knapsack)
                test_selection[i] = 0
                test_selection[k] = 1

            test_fitness, _, _ = calculate_fitness(test_selection, VALUES, WEIGHTS, CONSTRAINTS, PENALTY)

            if test_fitness > best_neighbor_fitness:
                best_neighbor_fitness = test_fitness
                best_neighbor_selection = test_selection.copy()
                improvement_found = True

        # Move to the best sampled neighbor
        if improvement_found and best_neighbor_fitness > current_fitness:
            current_selection = best_neighbor_selection
            current_fitness = best_neighbor_fitness
        else:
            break # No improvement found in the sampled neighborhood

    return current_selection, current_fitness

# -----------------------------------------------------------------------------------------------------------------

import time
# EXECUTION
start_time = time.time()

# Initial solution is the empty knapsack
initial_solution = np.zeros(NUM_ITEMS, dtype=int)

print("Starting Stochastic Local Search from an empty knapsack...")

final_solution, final_fitness = stochastic_hill_climbing(
    initial_solution, VALUES, WEIGHTS, CONSTRAINTS, PENALTY,
    MAX_LOCAL_SEARCH_ITERATIONS, SAMPLE_SIZE
)

end_time = time.time()
print(f"Total Heuristic Execution Time: {end_time - start_time:.2f} seconds")
print("-" * 70)

# ---------------------------------------------------------------------------------------------------------
# FINAL RESULTS

final_fitness, final_value, final_weights = calculate_fitness(
    final_solution, VALUES, WEIGHTS, CONSTRAINTS, PENALTY
)
selected_item_indices = np.where(final_solution == 1)[0]

print("--- Overall Best Solution Found (Stochastic Local Optimum) ---")

if final_fitness < final_value:
    print(f" **INFEASIBLE SOLUTION**: Constraints were violated (Fitness < Value).")
    print(f"   Violation is penalized, true value is {final_value}.")
else:
    print(" **FEASIBLE SOLUTION**.")

print(f"Final Total Value: {final_value} ")
print(f"Selected Item Count: {len(selected_item_indices)}")
print(f"Final Total Weights (First 5): {final_weights[:5]}...")
print(f"Knapsack Constraints (First 5): {CONSTRAINTS[:5]}...")

Items: 5000, Dimensions: 100
Knapsack Constraints (First 5): [35522 65218 54880 92359 88142]...
Neighborhood Sample Size: 5,000
----------------------------------------------------------------------
Starting Stochastic Local Search from an empty knapsack...
Total Heuristic Execution Time: 13.18 seconds
----------------------------------------------------------------------
--- Overall Best Solution Found (Stochastic Local Optimum) ---
 **FEASIBLE SOLUTION**.
Final Total Value: 24758 
Selected Item Count: 25
Final Total Weights (First 5): [12834 10925 13026 13994 11240]...
Knapsack Constraints (First 5): [35522 65218 54880 92359 88142]...
